<a href="https://colab.research.google.com/github/hildermesmedeiros/Remove-Video-Audio-Noise/blob/main/Remove_Video_Audio_Noise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas

## De linux

In [1]:
!apt-get update
!apt-get install -y autoconf automake libtool sox ffmpeg

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,858 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [1,084 kB]
Get:13 http://archive.ubuntu.com/ubuntu 

## Python

In [2]:
from google.colab import files
import pathlib

# Main

## Links:
#### [RNNoise](https://jmvalin.ca/demo/rnnoise/)
#### [RNNoise gitlab](https://gitlab.xiph.org/xiph/rnnoise/)

Licensa BSD 3


Vamos usar o modelo RNNoise, para gerar um notebook simples e comodo de remover barulho de fundo de nossos vídeos

In [3]:
# Clone RNNoise repository
!git clone https://gitlab.xiph.org/xiph/rnnoise.git
%cd rnnoise

# Compile RNNoise
!./autogen.sh
!./configure --enable-x86-rtcd
!make
%cd ..

Cloning into 'rnnoise'...
remote: Enumerating objects: 805, done.
remote: Counting objects: 100% (378/378), done.
remote: Compressing objects: 100% (154/154), done.
remote: Total 805 (delta 210), reused 372 (delta 208), pack-reused 427
Receiving objects: 100% (805/805), 1014.54 KiB | 3.55 MiB/s, done.
Resolving deltas: 100% (433/433), done.
/content/rnnoise
--2024-05-31 19:25:45--  https://media.xiph.org/rnnoise/models/rnnoise_data-4ac81c5c0884ec4bd5907026aaae16209b7b76cd9d7f71af582094a2f98f4b43.tar.gz
Resolving media.xiph.org (media.xiph.org)... 54.39.18.101, 2607:5300:203:2965::
Connecting to media.xiph.org (media.xiph.org)|54.39.18.101|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 22270507 (21M) [application/x-gzip]
Saving to: ‘rnnoise_data-4ac81c5c0884ec4bd5907026aaae16209b7b76cd9d7f71af582094a2f98f4b43.tar.gz’

rnnoise_data-4ac81c 100%[===================>]  21.24M  44.6MB/s    in 0.5s    

2024-05-31 19:25:46 (44.6 MB/s) - ‘rnnoise_data-4ac81c5c0884ec4

In [4]:
# @title Upload do seu Arquivo
your_video = files.upload()

Saving WhatsApp Video 2024-05-31 at 15.33.25.mp4 to WhatsApp Video 2024-05-31 at 15.33.25.mp4


In [5]:
for filename in your_video.keys():
    print(filename)

WhatsApp Video 2024-05-31 at 15.33.25.mp4


In [8]:
# @title Variaveis do projeto
input_data = f'"/content/{filename}"'
input_name_without_extension = pathlib.Path(input_data).stem
input_extension = pathlib.Path(input_data).suffix.strip('"').strip("'")
extracted_audio = "extracted_audio.wav"
extracted_audio_raw = "extracted_audio.raw"
denoised_audio_raw = "denoised_audio.raw"
encoded_denoised_audio = "encoded_denoised_audio.wav"

In [9]:
def check_file_type(input_extension: str) -> str:
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.flv', '.wmv']
    audio_extensions = ['.mp3', '.wav', '.aac', '.flac', '.ogg', '.wma']

    file_extension = input_extension.lower()

    if f'{file_extension}' in video_extensions:
        return "video"
    elif f'{file_extension}' in audio_extensions:
        return "audio"
    else:
        return "unknown"

class FileTypes:
    video = 'video'
    audio = 'audio'

In [10]:
file_type = check_file_type(input_extension)
file_type

'video'

In [11]:
output_file = f"output{input_extension}"
output_file

'output.mp4'

In [12]:
# @title Extraindo audio
if file_type == FileTypes.video:
    print('----Extracting audio from video----\n\n')
    !ffmpeg -i {input_data} -q:a 0 -map a {extracted_audio}
    !file {extracted_audio}
elif file_type == FileTypes.audio:
    print('----Audio file----\n\n')
    !ffmpeg -i {input_data} -q:a 0 -map a {extracted_audio}
    !file {extracted_audio}
else:
    raise ValueError()

----Extracting audio from video----


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --e

In [15]:
# @title Convertendo audio para formato raw
# Convert extracted audio to RAW format
!ffmpeg -i {extracted_audio} -f s16le -ar 48000 -ac 1 {extracted_audio_raw}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [16]:
!file {extracted_audio_raw}

extracted_audio.raw: data


In [17]:
# @title Extraindo ruído
# Apply RNNoise to remove noise
!/content/rnnoise/examples/rnnoise_demo extracted_audio.raw {denoised_audio_raw}

In [18]:
!file {denoised_audio_raw}

denoised_audio.raw: data


In [19]:
# @title Convertendo de raw para formato wav
# Convert denoised RAW audio back to WAV format
!ffmpeg -f s16le -ar 48000 -ac 1 -i {denoised_audio_raw} {encoded_denoised_audio}

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [20]:
!file {encoded_denoised_audio}

encoded_denoised_audio.wav: RIFF (little-endian) data, WAVE audio, Microsoft PCM, 16 bit, mono 48000 Hz


In [21]:
# @title Convertendo para o formato original
if file_type == FileTypes.video:
    print('----Extracting audio from video----\n\n')
    !ffmpeg -i {input_data} -i {encoded_denoised_audio} -c:v copy -map 0:v:0 -map 1:a:0 {output_file}
    !file {output_file}
elif file_type == FileTypes.audio:
    print('----Audio file----\n\n')
    !ffmpeg -i {input_data} {output_file}
    !file {output_file}
else:
    raise ValueError()

----Extracting audio from video----


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --e

In [23]:
# @title Baixando o arquivo final

files.download(output_file)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>